In [122]:
using CSV, DataFrames, Random, LinearAlgebra, Distances, Distributions, SpecialFunctions, Plots

#Single population model

#Each timestep, following processes are simulated in a given sequence
#1. Mortality of adults due to selection- Prob of death= (exp((T-T(opt))^2)/sigma)
#2. Reproduction-Mating between different genotypes represented by Barton's hypergeometric model
#3. Settlement- density dependent removal of newly produced larvae 

#Quantitative genetic model: 
#the 'heat tolerence' trait is determined by a total of L bialleilic loci, where allele "0" means
#unfavorable trait for temp. tolerence. All loci contribute independently (no linkage) and 
#equally. Therefore, the trait value of an individual is simply assumed to be the sum of all "1"
#alleles present in an individual. Approx. hypergeometric model for diploids is used 
#to describe the effect of evolutionary processes on trait propagation (barton 1992).

#Reproduction: a)Mating: A fraction of the population is assumed to reproduce asexually. The 
#rest of the population is assumed to start the mating process simultaneously where they 
#release their gametes in a pool and random mating is assumed to occur.

#Model parameters

#no. of loci
n=114
geno= collect(1:(2*n+1))
nt=length(geno)
#Growth rate
r=0.38
#Carrying capacity
K=1000

#Precalculate the probabilities of getting an offspring with trait k when the gametes with 'trait 
#value' i an j are combined

G=zeros(Float64,n+1,n+1,n+1)

for i in 0:n, j in 0:i, k in max(0,(i+j-n)):min(n,(i+j))
            m=collect(0:min(j,k,i+j-k))
            G[1+i,1+j,1+k]=sum(pdf.(Hypergeometric(i,n-i,j),m).*pdf.(Binomial.(i+j .- (2 .* m)),k .- m))
end
    
for k in 0:n
    G[:,:,1+k]=G[:,:,1+k]+transpose(G[:,:,1+k])
    for i1 in 0:n
        G[i1+1,i1+1,k+1] /= 2
    end
end
    
ind_haplR=zeros(Float64,2*n+1, 2*n+1)

for k in 0:n
    for i in 0:n
         ind_haplR[1+i,1+k] = G[1+i,1,1+k]
        for j in 0:n
            ind_haplR[1+j+n,1+k]=G[1+n,1+j,1+k]
        end
    end
end

R=zeros(Float64,nt,nt,nt)

for i in 0:(2*n), j in 0:(2*n), q in 0:(2*n)
     R[1+i,1+j,1+q]= sum(ind_haplR[1+i,1 .+ (0:q)] .* 
                         ind_haplR[1+j,1+q .- (0:q)])
end
    

#Start the simulation
    
res=zeros(77,nt)    
    
N=rand(Uniform(0,1.0),nt)
Ng0= N ./ sum(N)
Np0= 950 .* Ng0 

    
Ngen=deepcopy(Ng0)
Np=deepcopy(Np0)
 
#Some token trend of changes in optimal trait with time.    
topts=collect(range(30,stop=35,length=77))
    
for t in 1:77
                
        topt=topts[t]
        
        #Selection event
            
            probsurv=exp.(-((topt .-geno) .^2) ./ 50)
            Np=Np.*probsurv
            
        #Reproduction event
    
        newgen=zeros(Float64,nt)
        
        probs=Ngen*Ngen'
        
        for i in 1:length(Ngen)
            
            newgen[i]=sum(probs.*R[:,:,i])
        end
        
        #Settlement
        
        Np=(sum(Np)+ (r*sum(Np)*(1-(sum(Np)/K)))) .* newgen 
        
        Ngen=Np ./sum(Np)
        
        res[t,:]=Np


end 

    
